In [23]:
import sys
#!conda install --yes -c anaconda sqlalchemy

In [24]:
from fastapi import FastAPI
from fastapi.testclient import TestClient
from sqlalchemy import create_engine
import psycopg2 as pg
app = FastAPI()
portNum = 5902

In [15]:
@app.get("/cities")
def read_cities():
    with pg.connect(dbname="main", user="aurimas", password="aurimas6242", host="localhost", port=portNum) as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT cityID, cityname FROM cities")
            data = cur.fetchall()
    citiesList = []
    for i in data:
        citiesList.append({"ID": i[0], "Name": i[1]})
    return citiesList

In [16]:
client = TestClient(app)
response = client.get("/cities")
print(response.json())

[{'ID': 1, 'Name': 'Atlanta'}]


In [17]:
@app.get("/pois")
def read_POIs(city, type):
    with pg.connect(dbname="main", user="aurimas", password="aurimas6242", host="localhost", port=portNum) as conn:
        with conn.cursor() as cur:
            sql = "SELECT poiid, name, lat, long FROM cityh3map inner join (SELECT * FROM pois WHERE category = '" + type + "') pois ON cityh3map.h3id = pois.h3id WHERE cityh3map.cityid = " + str(city)
            cur.execute(sql)
            data = cur.fetchall()
    response = []
    for i in data:
        response.append({"ID": i[0], "Name": i[1], "Coordinates": {"lat": i[2], "long": i[3]}})
    return response

In [18]:
client = TestClient(app)
response = client.get("/pois?city=1&type=Restaurants")
print(response.json())

[{'ID': 311443, 'Name': 'Sweet Monkey', 'Coordinates': {'lat': 33.801186, 'long': -84.327446}}, {'ID': 311442, 'Name': 'Varuni Napoli', 'Coordinates': {'lat': 33.79656, 'long': -84.36876}}, {'ID': 311441, 'Name': 'China Doll', 'Coordinates': {'lat': 33.922203, 'long': -84.46852}}, {'ID': 311440, 'Name': 'Zucca Bar & Pizzeria', 'Coordinates': {'lat': 33.883747, 'long': -84.51473}}, {'ID': 311439, 'Name': 'Chopsticks China Bistro', 'Coordinates': {'lat': 33.977592, 'long': -84.44023}}, {'ID': 311438, 'Name': "Capozzi's Pizza", 'Coordinates': {'lat': 33.98661, 'long': -84.42453}}, {'ID': 311437, 'Name': "Capozzi's New York Deli", 'Coordinates': {'lat': 33.98655, 'long': -84.42457}}, {'ID': 311436, 'Name': 'The Marietta Local', 'Coordinates': {'lat': 33.952103, 'long': -84.54888}}, {'ID': 311435, 'Name': 'Piastra', 'Coordinates': {'lat': 33.953007, 'long': -84.550415}}, {'ID': 311434, 'Name': 'Taqueria Tsunami', 'Coordinates': {'lat': 33.95214, 'long': -84.54972}}, {'ID': 311433, 'Name': '

In [19]:
#To do: accessibility
@app.get("/demographics")
def read_demographics(city, category):
    with pg.connect(dbname="main", user="aurimas", password="aurimas6242", host="localhost", port=portNum) as conn:
        with conn.cursor() as cur:
            sql = "SELECT demo.h3id, groupname, sum(total) FROM (SELECT * FROM censush3map inner join "
            sql += "demographics ON censush3map.censusblockgroupid = demographics.censusblockgroupid) demo "
            sql += "inner join cityh3map on cityh3map.h3id = demo.h3id WHERE cityh3map.cityid = "
            sql += str(city) + " AND groupname = '" + category + "' GROUP BY demo.h3id, groupname"
            cur.execute(sql)
            data = cur.fetchall()
    dicData = {}
    for i in data:
        if(i[0] not in dicData):
            dicData[i[0]] = []
        dicData[i[0]].append([i[1], i[2]])
    res = {"HexID": [], "Demographics": []}
    for hexID in dicData:
        res["HexID"].append(hexID)
        for j in dicData[hexID]:
            res["Demographics"].append([{"Name": j[0], "Population": j[1]}])
    return res

In [20]:
client = TestClient(app)
response = client.get("/demographics?city=1&category=65 years and over female")
print(response.json())

{'HexID': ['8944c100907ffff', '8944c100917ffff', '8944c100923ffff', '8944c100927ffff', '8944c10092fffff', '8944c100933ffff', '8944c100937ffff', '8944c10093bffff', '8944c100983ffff', '8944c100987ffff', '8944c10098bffff', '8944c10098fffff', '8944c100993ffff', '8944c100997ffff', '8944c10099bffff', '8944c1009a3ffff', '8944c1009a7ffff', '8944c1009abffff', '8944c1009afffff', '8944c1009b3ffff', '8944c1009b7ffff', '8944c1009bbffff', '8944c100d03ffff', '8944c100d07ffff', '8944c100d0bffff', '8944c100d0fffff', '8944c100d13ffff', '8944c100d17ffff', '8944c100d1bffff', '8944c100d23ffff', '8944c100d27ffff', '8944c100d2bffff', '8944c100d2fffff', '8944c100d33ffff', '8944c100d37ffff', '8944c100d3bffff', '8944c100d67ffff', '8944c100d73ffff', '8944c100d77ffff', '8944c100d83ffff', '8944c100d87ffff', '8944c100d8bffff', '8944c100d8fffff', '8944c100d93ffff', '8944c100d97ffff', '8944c100d9bffff', '8944c100da3ffff', '8944c100da7ffff', '8944c100dabffff', '8944c100dafffff', '8944c100db3ffff', '8944c100db7ffff', '

In [25]:
#To do
@app.get("/catchment")
async def read_catchment(city, poi, TimeofDay, distance, category):
    with pg.connect(dbname="main", user="aurimas", password="aurimas6242", host="localhost", port=portNum) as conn:
        with conn.cursor() as cur:
            sqlDemo = "SELECT demo.h3id, groupname, total FROM (SELECT * FROM censush3map inner join "
            sqlDemo += "demographics ON censush3map.censusblockgroupid = demographics.censusblockgroupid) demo "
            sqlDemo += "inner join cityh3map on cityh3map.h3id = demo.h3id WHERE cityh3map.cityid = "
            sqlDemo += str(city) + " AND groupname = '" + category + "'"
            sql = "(SELECT * FROM ((SELECT * FROM catchmenth3map inner join "
            sql += "(SELECT catchmentid as catchmentid2, originh3id, timeofday, "
            sql += "timedistance, geometry from catchments"
            sql += " WHERE timeofday = '" + TimeofDay + "' AND timedistance = " + str(distance) + ") catchments"
            sql += " ON catchments.catchmentid2 = catchmenth3map.catchmentid) catchment"
            sql += " inner join (SELECT id as id2, cityid, h3id AS h3id2 from cityh3map"
            sql += " WHERE cityid = " + str(city) + ") cityh3map on cityh3map.h3id2"
            sql += " = catchment.h3id))"
            sqlFinal = "SELECT catchment.catchmentid, catchment.geometry, groupname, total FROM (" + sql + ") catchment INNER JOIN ("
            sqlFinal += sqlDemo + ") demo ON demo.h3id = catchment.h3id"
            print(sqlFinal)
            cur.execute(sqlFinal)
            data = cur.fetchall()
    return data

In [ ]:
client = TestClient(app)
response = client.get("/catchment?city=1&poi=1&TimeofDay=morning&distance=30&category=65 years and over female")
print(len(response.json()))

SELECT catchment.catchmentid, catchment.geometry, groupname, total FROM ((SELECT * FROM ((SELECT * FROM catchmenth3map inner join (SELECT catchmentid as catchmentid2, originh3id, timeofday, timedistance, geometry from catchments WHERE timeofday = 'morning' AND timedistance = 30) catchments ON catchments.catchmentid2 = catchmenth3map.catchmentid) catchment inner join (SELECT id as id2, cityid, h3id AS h3id2 from cityh3map WHERE cityid = 1) cityh3map on cityh3map.h3id2 = catchment.h3id))) catchment INNER JOIN (SELECT demo.h3id, groupname, total FROM (SELECT * FROM censush3map inner join demographics ON censush3map.censusblockgroupid = demographics.censusblockgroupid) demo inner join cityh3map on cityh3map.h3id = demo.h3id WHERE cityh3map.cityid = 1 AND groupname = '65 years and over female') demo ON demo.h3id = catchment.h3id


In [ ]:
#To do
@app.get("/city_stats")
async def read_city_stats(city, category, timeofDay, distance, type):
    return {"1": "Hello World"}